In [ ]:
!pip install pinecone

In [ ]:
!pip install -U pinecone_notebooks

In [ ]:
import os
from pinecone_notebooks.colab import Authenticate

if not os.environ.get("pcsk_346CtC_P7GPEDx1WxTNeaGgzVeMmDwBsdoFDFLTrTNuV3swxcGHPRT5dVxyNHQFcWHrvjM"):
    Authenticate()

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
api_key = os.environ.get("pcsk_346CtC_P7GPEDx1WxTNeaGgzVeMmDwBsdoFDFLTrTNuV3swxcGHPRT5dVxyNHQFcWHrvjM")
pc = Pinecone(api_key=api_key)

In [ ]:
!pip install --upgrade --quiet langchain pypdf sentence-transformers

In [ ]:
!pip install PyPDF2

In [ ]:
from pathlib import Path
import os
from PyPDF2 import PdfReader

# Set the file path
pdf_path = Path("/budget_speech.pdf")

# Load the PDF
reader = PdfReader(str(pdf_path))

# Extract text
text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview the first 1000 characters


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_text(text)
print(f"Split into {len(texts)} chunks")


In [ ]:
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = embedder.encode(texts, show_progress_bar=True)


In [ ]:
from pinecone import Pinecone, ServerlessSpec

if 'budget-speech-latest-rag' not in pc.list_indexes().names():
      pc.create_index(
          name='budget-speech-latest-rag',
          dimension=384,
          metric='cosine',
          spec=ServerlessSpec(
              cloud='aws',
              region='us-east-1'
          )
      )

In [ ]:
index = pc.Index('budget-speech-latest-rag')
vectors = [(f"id-{i}", embeddings[i], {"text": texts[i]}) for i in range(len(texts))]
index.upsert(vectors, namespace="budget-speech-latest-rag")


In [ ]:
query = "explain the philosophy, Sabka Saath Sabka Vikas"
query_embedding = embedder.encode([query])[0]

# Search in Pinecone
results = index.query(vector=query_embedding.tolist(), top_k=1, include_metadata=True)

# Display retrieved texts
for match in results['matches']:
    print(match['metadata']['text'], '\n' + '-'*80)


In [ ]:
import nbformat

def clean_notebook(path):
    with open(path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    for cell in nb.cells:
        if 'outputs' in cell:
            cell['outputs'] = []
        if 'execution_count' in cell:
            cell['execution_count'] = None

    with open(path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

clean_notebook("budget_speech_latest.ipynb")
